In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob2 import glob  
import pandas_profiling
from sklearn.preprocessing import RobustScaler

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

In [24]:
data_list = glob('./aT_data/data/train/*.csv')

In [15]:
df = pd.read_csv(data_list[0])
df

,datadate,단가(원),거래량,거래대금(원),경매건수,도매시장코드,도매법인코드,산지코드,해당일자_전체평균가격(원),해당일자_전체거래물량(kg),...,월구분_12월,월구분_1월,월구분_2월,월구분_3월,월구분_4월,월구분_5월,월구분_6월,월구분_7월,월구분_8월,월구분_9월
0,20130101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
1,20130102,1298.70,435.0,566065.50,3.0,350301.0,35030101.0,561.0,1587.314723,116539.0,...,0,1,0,0,0,0,0,0,0,0
2,20130103,929.07,1005.0,936562.50,5.0,350402.0,35040202.0,580.0,1731.765036,812663.0,...,0,1,0,0,0,0,0,0,0,0
3,20130104,960.96,360.0,346653.00,4.0,340101.0,34010102.0,330.0,1433.421743,282245.0,...,0,1,0,0,0,0,0,0,0,0
4,20130105,720.72,5400.0,3903900.00,1.0,311201.0,31120103.0,467.0,1549.518789,347766.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,20161227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
1457,20161228,3663.66,1125.0,4121117.00,7.0,380201.0,38020102.0,680.0,3666.548045,237547.0,...,1,0,0,0,0,0,0,0,0,0
1458,20161229,2507.49,30.0,75315.24,1.0,311201.0,31120103.0,471.0,3652.588226,118475.0,...,1,0,0,0,0,0,0,0,0,0
1459,20161230,3793.79,1245.0,4713282.00,15.0,250001.0,25000102.0,314.0,3578.237741,83490.0,...,1,0,0,0,0,0,0,0,0,0


In [33]:
def feature_select(df):
       #시간 코사인 인코딩
       if df['datadate'].all() == 0:
         df['datadate'] = 20180520
       df['datetime'] = pd.to_datetime(df['datadate'], format="%Y%m%d")
       df['cos_time'] = np.cos(2*np.pi*df['datetime'].dt.dayofyear/365)

       # 결측치
       df = df.replace(' ', np.nan)

       # 주산지 초기온도, 최대온도, 최저온도, 평균온도, 강수량, 습도
       df['주산지_초기온도'] = (df['주산지_0_초기온도(℃)'] + df['주산지_1_초기온도(℃)'] + df['주산지_2_초기온도(℃)'])/3
       df['주산지_최대온도'] = (df['주산지_0_최대온도(℃)'] + df['주산지_1_최대온도(℃)'] + df['주산지_2_최대온도(℃)'])/3
       df['주산지_최저온도'] = (df['주산지_0_최저온도(℃)'] + df['주산지_1_최저온도(℃)'] + df['주산지_2_최저온도(℃)'])/3
       df['주산지_평균온도'] = (df['주산지_0_평균온도(℃)'] + df['주산지_1_평균온도(℃)'] + df['주산지_2_평균온도(℃)'])/3
       df['주산지_강수량'] = (df['주산지_0_강수량(ml)'] + df['주산지_1_강수량(ml)'] + df['주산지_2_강수량(ml)'])/3

       # 필요한 컬럼 선택
       
       market_cols = ['단가(원)', '거래량'] # '경매건수',, '거래대금(원)'

       market_code_cols = [] # '도매시장코드', '도매법인코드', '산지코드 '

       market_statistics_cols = [ '일자별_도매가격_평균(원)', '해당일자_전체거래물량(kg)'
                                 ] 
       # '상위가격 평균가(원)',  '하위가격 평균가(원)',
       # '일자별_소매가격_최대(원)', '일자별_소매가격_평균(원)', '일자별_소매가격_최소( 원)',
       #  '하위가격 거래물량(kg)', '상위가격 거래물량(kg)',
       #  '일자별_도매가격_최대(원)','일자별_도매가격_최소(원)', , 
       
       export_cols = [] # '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)'
       
       date_cols = ['cos_time'] 
       # '일자구분_중순', '일자구분_초순', '일자구분_하순', '월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월','월구분_3월', '월구분_4월', '월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월', 'datetime'

       weather_cols = ['주산지_평균온도', '주산지_강수량'] 
       # '주산지_초기온도', '주산지_최대온도', '주산지_최저온도', 
      #  '주산지_0_초기온도(℃)', '주산지_1_초기온도(℃)', '주산지_2_초기온도(℃)',
      #  '주산지_0_최대온도(℃)', '주산지_1_최대온도(℃)', '주산지_2_최대온도(℃)',
      #  '주산지_0_최저온도(℃)', '주산지_1_최저온도(℃)', '주산지_2_최저온도(℃)',
      #  '주산지_0_평균온도(℃)', '주산지_1_평균온도(℃)', '주산지_2_평균온도(℃)',
      #  '주산지_0_강수량(ml)', '주산지_1_강수량(ml)', '주산지_2_강수량(ml)',
      #  '주산지_0_습도(%)', '주산지_1_습도(%)', '주산지_2_습도(%)'

       selected_columns = ['datadate', '해당일자_전체평균가격(원)']  + market_cols + market_code_cols + market_statistics_cols + export_cols +  date_cols + weather_cols # train 에서 사용하지 않는 열
       df = df[[c for c in df.columns if c in selected_columns]]

       return df

In [49]:
def feature_scale(df):
    scaler = RobustScaler()
    scale_columns = [c for c in df.columns if c not in ['datadate', 'cos_time']]
    
    df[scale_columns] = scaler.fit_transform(df[scale_columns])

    return df

In [56]:
def feature_na(df):
    df = df.interpolate()
    df = df.fillna(method = 'bfill')

    return df

In [57]:
df2 = feature_select(df)
df3 = feature_scale(df2)
df4 = feature_na(df3)
df4

,datadate,단가(원),거래량,해당일자_전체평균가격(원),해당일자_전체거래물량(kg),일자별_도매가격_평균(원),cos_time,주산지_평균온도,주산지_강수량
0,20130101,-0.441031,-0.424621,-0.343993,-0.357797,-0.410256,0.999852,-0.537634,-0.200000
1,20130102,-0.441031,-0.424621,-0.343993,-0.357797,-0.410256,0.999407,-0.344086,-0.200000
2,20130103,-0.752658,-0.049375,-0.194723,3.059190,-0.410256,0.998667,-0.526882,-0.200000
3,20130104,-0.725772,-0.473996,-0.503021,0.455586,-0.410256,0.997630,-0.503226,-0.200000
4,20130105,-0.928313,2.843976,-0.383050,0.777202,-0.346154,0.996298,-0.556989,0.351724
...,...,...,...,...,...,...,...,...,...
1456,20161227,1.465505,-0.197498,1.736683,-0.187111,1.687500,0.998667,-0.513978,0.724138
1457,20161228,1.552815,0.029625,1.804613,0.236182,1.683333,0.999407,-0.440860,1.110345
1458,20161229,0.578073,-0.691244,1.790187,-0.348294,1.683333,0.999852,-0.638710,0.000000
1459,20161230,1.662524,0.108624,1.713356,-0.520021,1.683333,1.000000,-1.247312,-0.089655


In [58]:
profile=df4.profile_report()
profile.to_file(output_file="pr.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]